In [1]:
import twitter
import tweepy
from tweepy import API
from tweepy import Cursor
import numpy as np
from sqlalchemy import create_engine
import pandas as pd
import time
from time import sleep
import os
import requests
from bs4 import BeautifulSoup
import facebook
import os

ck = os.environ.get('CK')
cs = os.environ.get('CS')
atk = os.environ.get('ATK')
ats = os.environ.get('ATS')

# print(os.environ.get(ck))

auth = tweepy.OAuthHandler(ck, cs)
auth.set_access_token(atk, ats)
auth_api = API(auth)

api = tweepy.API(auth, wait_on_rate_limit=True)

token = os.environ.get('fb')

In [5]:
# Get a LIST of friends that you are following
list_of_friends_id = api.friends_ids()
print(len(list_of_friends_id))
list_of_friends_id[:5]

59


[19980499, 14940354, 20065936, 15492359, 788524]

In [ ]:
# To get the ids of people that are following you
x = api.followers_ids()
print(len(x))
x[:5]

In [ ]:
# Getting informations from other users timelines
y = api.user_timeline(15754281,count=5)
print(type(y),len(y))

In [ ]:
print(y[0].geo,y[0].id,y[0].favorite_count,y[0].retweet_count,y[0].lang)

In [ ]:
# my id
my_id = api.me().id
len(api.user_timeline(my_id))

In [ ]:
# Getting my id and taking info from it
my_id = api.me().id
y = api.user_timeline(my_id)
my_tweets_list = [i.id for i in y]

In [ ]:
list_of_friends_id = api.friends_ids()

def theupdate_at_work():
    while True:
        df = pd.DataFrame(columns=np.arange(11))
        for user in list_of_friends_id[:5]:
            list_tweet = api.user_timeline(user,count=3,exclude_replies=True)
            for tweet in list_tweet:
                try:
                    hashtags = ' '.join([i for i in tweet._json['text'].split() if i[0]=='#'])
                    url=tweet._json['entities']['urls'][0]['url']
                    page = requests.get(url)
                    soup = BeautifulSoup(page.text, 'html.parser')
                    h1 = '#news '+soup.find('h1').text.strip()

                    df.loc[len(df)] = [tweet.id, #0
                                       tweet.geo, #1
                                       tweet.favorite_count, #2
                                       tweet.retweet_count, #3
                                       tweet.lang, #4
                                        user,  #5
                                       tweet._json['user']['id'],  #6
                                       tweet._json['text'],  #7
                                       tweet._json['entities']['urls'][0]['url'],   #8
                                       hashtags,  #9
                                       h1+' '+url+' '+hashtags]        #10
                except:
                    pass
        df[11]=df[2]+df[3]*2   #scoring of the tweets
        df = df[df[4]=='en']
        df[12] = [False if i[:2]=='RT' else True for i in df[7]]
        df = df.sort_values(by=11,ascending=False)
        df = df[df[12]]
        counter = 0
        counter_tweet = 0
        counter_errors = 0
        for i in list(df.index):
            # counter+=1
            # print('Global counter: ',counter, end='\r')
            if counter_tweet==5:
                break
            else:
                try:
                    api.update_status(df.loc[i][10])
                    fb = facebook.GraphAPI(access_token=token)
                    fb.put_object(
                        parent_object="me",
                        connection_name="feed",
                        message=df.loc[i][7],
                        link=df.loc[i][8]
                                 )
                    counter_tweet+=1
                    #print('tweet number: ',counter_tweet,end='\r')
                    for second in range(180):
                        time.sleep(1)
                        #print('timer: ',179-second,end='\r')
                except:
                    pass

In [ ]:
theupdate_at_work()

In [4]:
df = pd.DataFrame(columns=np.arange(11))
for user in list_of_friends_id[:5]:
    list_tweet = api.user_timeline(user,count=3,exclude_replies=True)
    for tweet in list_tweet:
        hashtags = ' '.join([i for i in tweet._json['text'].split() if i[0]=='#'])
        url=tweet._json['entities']['urls'][0]['url']
        page = requests.get(url)
        soup = BeautifulSoup(page.text, 'html.parser')
        h1 = '#news '+soup.find('h1').text.strip()
        df.loc[len(df)] = [tweet.id, #0
                           tweet.geo, #1
                           tweet.favorite_count, #2
                           tweet.retweet_count, #3
                           tweet.lang, #4
                            user,  #5
                           tweet._json['user']['id'],  #6
                           tweet._json['text'],  #7
                           tweet._json['entities']['urls'][0]['url'],   #8
                           hashtags,  #9
                           h1+' '+url+' '+hashtags]        #10
df[11]=df[2]+df[3]*2   #scoring of the tweets
df = df[df[4]=='en']
df[12] = [False if i[:2]=='RT' else True for i in df[7]]
df = df.sort_values(by=11,ascending=False)
df = df[df[12]]

NameError: name 'list_of_friends_id' is not defined

In [ ]:
tweet = api.user_timeline(15754281,count=3,exclude_replies=True)

In [ ]:
tweet[0]._json['entities']['urls'][0]

In [ ]:
tweet[0]._json['entities']['urls'][0]['url']

In [ ]:
import selenium

In [ ]:
df.shape

In [ ]:
url='https://t.co/Lf6AqowKyN'
page = requests.get(url)
soup = BeautifulSoup(page.text, 'html.parser')

In [ ]:
'#news '+soup.find('h1').text.strip()

In [ ]:
api.update_status(('#news '+soup.find('h1').text.strip()+' '+url))

In [6]:
df = pd.DataFrame(columns=np.arange(11))
for user in list_of_friends_id[:5]:
    list_tweet = api.user_timeline(user,count=3,exclude_replies=True)
    for tweet in list_tweet:
        try:
            hashtags = ' '.join([i for i in tweet._json['text'].split() if i[0]=='#'])
            url=tweet._json['entities']['urls'][0]['url']
            page = requests.get(url)
            soup = BeautifulSoup(page.text, 'html.parser')
            h1 = '#news '+soup.find('h1').text.strip()

            df.loc[len(df)] = [tweet.id, #0
                               tweet.geo, #1
                               tweet.favorite_count, #2
                               tweet.retweet_count, #3
                               tweet.lang, #4
                                user,  #5
                               tweet._json['user']['id'],  #6
                               tweet._json['text'],  #7
                               tweet._json['entities']['urls'][0]['url'],   #8
                               hashtags,  #9
                               h1+' '+url+' '+hashtags]        #10
        except:
            pass
print(df.shape)
df[11]=df[2]+df[3]*2   #scoring of the tweets
df = df[df[4]=='en']
df[12] = [False if i[:2]=='RT' else True for i in df[7]]
df = df.sort_values(by=11,ascending=False)
df = df[df[12]]

(11, 11)


In [16]:
df.index

Int64Index([7, 6, 0, 10, 5, 2, 4, 1, 8, 3], dtype='int64')

In [17]:
df.head(2)

,0,1,2,3,4,5,6,7,8,9,10,11,12
7,1136659216838668289,None,381,139,en,15492359,15492359,These incredible disappearing creatures use th...,https://t.co/Ph4UoK2YQR,,#news TED TalksVerifizierter Account https://t...,659,True
6,1136663809656053760,None,307,61,en,15492359,15492359,The delightful science of baby laughter: https...,https://t.co/MBuw2yoWCJ,,#news Meet a scientist with a most delightful ...,429,True


In [18]:
df.loc[7][10]

'#news TED TalksVerifizierter Account https://t.co/Ph4UoK2YQR '

In [7]:
df[10][0]

'#news ATP TourVerifizierter Account https://t.co/w162qdMUTP '

In [8]:
np.arange(10)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [ ]:
' '.join([i for i in df[7][23].split() if i[0]=='#'])

In [ ]:
df.head(2)